**SQL**

**Цель исследования** проанализировать базу данных для последующего формирования ценностных предложений для новых продуктов.

**Источник данных** - база данных с информацией о книгах, издательствах, авторах, пользовательских обзорах книг.

**Задания**:
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# 1. **Настройка доступа к базе данных**

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


# 2. **Изучение общей информации о данных**

In [4]:
def select(query):
    return pd.io.sql.read_sql(query, con = engine) 

In [5]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table_name in tables:
    print('Таблица', table_name)
    query = 'SELECT * FROM {} LIMIT 5'.format(table_name)
    query1 = 'SELECT COUNT (*) FROM {}'.format(table_name)
    display(select(query))
    display('Количество строк в таблице {}'.format(table_name), select(query1))
    display(select('select * from {}'.format(table_name)).info())

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


'Количество строк в таблице books'

,count
0,1000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


...

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


'Количество строк в таблице reviews'

,count
0,2793


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

В таблице books содержится информация об id книги, id автора, id издательства, название книги, дата публикации и кол-во страниц.
В таблице books 6 колонок, 1000 строк, пропусков нет.

В таблице authors содержится информация об id автора и его имени(псевдониме).
В таблице authors 2 колонки, 636 строк, пропусков нет.

В таблице publishers содержится информация об id издательства и его названии.
В таблице publishers 2 колонки, 340 строк, пропусков нет.

В таблице ratings содержится информация об id оценки, id книги, имя пользователя, оставившего оценку и оценка книги.
В таблице ratings 4 колонки, 6456 строки, пропусков нет.

В таблице reviews содержится информация об id обзора, id книги, имя пользователя, написавшего обзор, текст обзора.
В таблице reviews 4 колонки, 2793 строк, пропусков нет.


# 3. **Посчитайте, сколько книг вышло после 1 января 2000 года**

In [7]:
select('''SELECT COUNT (book_id) FROM books
        WHERE publication_date > '2000-01-01'::date''')

,count
0,819


После 1 января 2000 года было опубликовано 819 книг.

# 4. **Для каждой книги посчитайте количество обзоров и среднюю оценку**

In [8]:
select('''SELECT b.book_id,
                 b.title,
                 COUNT(DISTINCT rv.review_id) AS count_reviews,
                 AVG(rt.rating) AS avg_rating
        FROM books AS b
        LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
        ORDER BY count_reviews DESC, avg_rating DESC
        ''')

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


In [9]:
select('''SELECT b.book_id,
                 b.title,
                 COUNT(DISTINCT rv.review_id) AS count_reviews,
                 AVG(rt.rating) AS avg_rating
        FROM books AS b
        LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
        ORDER BY avg_rating DESC
        ''')

,book_id,title,count_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


В базе содержатся книги, на которые написаны 7 рецензий и на которые нет ни одной рецензии. Средние рейтинги книг находятся в пределах 1,5 - 5,0 баллов.

# 5. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
select('''SELECT publisher, COUNT(b.book_id)
            FROM publishers AS p
            JOIN books AS b ON p.publisher_id = b.publisher_id
            WHERE b.num_pages > 50
            group by p.publisher_id
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1
        ''')

,publisher,count
0,Penguin Books,42


Издательство Penguin Books выпустило больше всего книг, толще 50 страниц - 42 штуки.

# 6. **Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [11]:
select('''
            SELECT a.author, AVG(rt.rating) AS avg_rating
            FROM authors AS a
            JOIN books AS b ON a.author_id=b.author_id
            LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
            WHERE b.book_id IN
                    (SELECT b.book_id
                     FROM books AS b
                     JOIN ratings AS rt ON b.book_id=rt.book_id
                     GROUP BY b.book_id
                     HAVING COUNT(rt.rating_id)>=50)
            GROUP BY a.author
            ORDER BY avg_rating DESC
            LIMIT 1
            ''')


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling имеет самую высокую среднюю оценку книг - 4,287 (среди книг с более чем 50 оценками).

# 7. **Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок**

In [12]:
select('''SELECT t.count_review / t.count_username AS avg_review
        FROM (SELECT COUNT(review_id) AS count_review, COUNT(DISTINCT username) AS count_username
        FROM reviews
        WHERE username IN 
        (SELECT username
            FROM ratings
            GROUP BY username
            HAVING COUNT (rating_id) > 48
            ORDER BY COUNT (rating_id) DESC)) AS t
            ''')

,avg_review
0,24


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24 обзора.

# 8. **Вывод**

Было выполнено 5 запросов к БД, результатами которых стали следующие данные:
1. После 1 января 2000 года было опубликовано 819 книг.
2. В базе содержатся книги, на которые написаны 7 рецензий и на которые нет ни одной рецензии. Средние рейтинги книг находятся в пределах 1,5 - 5,0 баллов.
3. Издательство Penguin Books выпустило больше всего книг, толще 50 страниц - 42 штуки.
4. J.K. Rowling имеет самую высокую среднюю оценку книг - 4,287 (среди книг с блее чем 50 оценками). 
5. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора.
